* Think of a word or words that define each point  
    * Proximity to each of those words creates a new centroid.
    * proximity is determined using pre-trained word2vec
    * basically, check how close a new word is to each of the centroids (which are defined by words)
    * how close it is to each centroid contributes contributes to it's distance in that direction

In [1]:
import pandas as pd

import pickle

from scipy.spatial import minkowski_distance

Find songs closest to each centroid and define those moods

In [2]:
pickle_in = open('../Data/cluster_centers_tfidf_sent_rm_pca_9', 'rb')
centroids = pickle.load(pickle_in)
pickle_in.close()

In [3]:
df = pd.read_pickle('../Data/labeled_points_tfidf_sent_rm_pca_9')

In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [5]:
features = df.select_dtypes(include=numerics)
no_features = df.select_dtypes(exclude=numerics)

In [6]:
centroids[0]

array([ 2.18473877, -0.66892578,  0.66449067, -0.35420632,  0.25238495,
        0.31659498,  0.37395208,  0.00336713,  0.21607491])

In [11]:
def match_sentiment(sentiment_df, feats_to_match):
    distances_df = sentiment_df.apply(lambda x:
                                      minkowski_distance(x.values,
                                                         feats_to_match, p=1),
                                      axis=1)
    distances_df = pd.DataFrame(distances_df)
    distances_df.columns = ['distance']
    n_smallest = distances_df.nsmallest(15, 'distance').index
    return n_smallest

In [15]:
n_smallest = match_sentiment(features.drop('labels', axis=1), centroids[5])

In [16]:
df[df.labels == 2].shape

(1835, 29)

In [17]:
for i in n_smallest:
    print(df.iloc[i].uri + ',',
          df.iloc[i].unaltered_artist_name + ',',
          df.iloc[i].unaltered_song_title,
          '\n')

spotify:track:2WQDKdkSAq0ks06xMvdUb3, Copeland, Strange Flower 

spotify:track:6KsysWbGoINWOIp9YV9D29, The Postmarks, Five Years 

spotify:track:0FxJQ04cqvczvRzZhwz0go, Blitzen Trapper, Reno 

spotify:track:3pazhNjcz3anIg0VxRDt8f, Great Lake Swimmers, Don't Leave Me Hanging 

spotify:track:3QinothXo6s9uhwCXlI8wp, The National, Slipped 

spotify:track:2vGgx4Lq8Vr3vYdbmm878k, The Weepies, Volunteer 

spotify:track:0dwE2D1Ehd6fSry5qfluRR, Luke Haines, Angry Man on Small Train 

spotify:track:7lSdUlVf8k6kxklKkskb1m, Lana Del Rey, Heroin 

spotify:track:3bCzZbJKfflQjAKjahVGOK, The Brunettes, Your Heart Dies 

spotify:track:6nEZy8UD85zJTmRWdbYith, Elf Power, The Demon's Daughter 

spotify:track:1ToDLudrcaboyoUJcnqJKz, Teenage Fanclub, Don't Hide 

spotify:track:77Q3cCu18PM7eqyg49DpfO, The Notwist, Casino 

spotify:track:2NVuUSpEzB7O3r3a6IyTye, The Helio Sequence, Broken Afternoon 

spotify:track:4B2nRNViRK1fZj4ShhfgSa, Conor Oberst, Lenders in the Temple 

spotify:track:10IOaf5bghHYfgB8Z65Yi

Pretty sure I'm using 12 centroids here...

Centroid descriptions:
* 0:
    * psychedelic 
    * raw
    * distant
    * trippy
    * weird
    * lost
    * on my own path
    * outside/wild
    * primal
    * carnal
    * appreciating simple things, simple life
    * country
    * acoustic
* 1:
    * agitated
    * frustrated
    * annoyed
    * very lost
    * mad at the world
    * sad x 2
    * alone x 2
    * sad 
    * lonely
* 2:
    * quiet
    * content
    * let go
    * etherial
    * regret
    * missing old times
    * I feel like I've lost someone
    * letting go
    * having trouble letting go
* 3:
    * no idea... skipping for now
* 4:
    * casual rebellious
    * generally feels pretty good
    * whiney...
    * not really sure
    * annoying rebel without a cause
* 5:
    * simple
    * pleasant
    * chill
    * slow
    * country-esque
    * melancholy
* 6:
    * relaxing
    * cheer me up
    * I don't wanna feel alone
* 7:
    * 

In [23]:
df[(df.artist_name == 'esthero') & (df.song_title == 'crash prelude')].lyrics.values

array(['crash floor hold breath fall apart made simple mistake paying heart speak bed afford lay remember always said leave alone cause wanna crash afraid going nowhere way fast hear say conversation past maybe brave much raising got somehow path paved keep leading wanna crash wanna crash'],
      dtype=object)